In [1]:
import os 

In [2]:
%pwd

'e:\\Summarizer Repositry\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Summarizer Repositry\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [6]:
from src.TextSummarizer.constants import *
from src.TextSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config


In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import LEDForConditionalGeneration, LEDTokenizer
from datasets import load_dataset, load_from_disk
import torch


d:\Apps\Anaconda\envs\textS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-02-18 15:43:49,230: INFO: config: PyTorch version 2.1.2 available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = LEDTokenizer.from_pretrained(self.config.model_ckpt)
        model_LED = LEDForConditionalGeneration.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model= model_LED)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_LED, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_LED.save_pretrained(os.path.join(self.config.root_dir,"LED-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e


[2024-02-18 15:43:49,698: INFO: common: yaml file: E:\Summarizer Repositry\Text-Summarizer-Project\config\config.yaml loaded successfully]
[2024-02-18 15:43:49,702: INFO: common: yaml file: E:\Summarizer Repositry\Text-Summarizer-Project\params.yaml loaded successfully]
[2024-02-18 15:43:49,704: INFO: common: created directory at: artifacts]
[2024-02-18 15:43:49,706: INFO: common: created directory at: artifacts/model_trainer]


  0%|          | 0/51 [00:00<?, ?it/s]Input ids are automatically padded from 171 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 295 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 29 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 63 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 141 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 202 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 106 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 31 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 356 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically p

{'loss': 2.923, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


Input ids are automatically padded from 236 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 76 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 105 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 191 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 122 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 74 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 218 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 60 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 431 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 363 to 1024 to be a multipl

{'loss': 2.7173, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.3526, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


Input ids are automatically padded from 315 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 221 to 1024 to be a multiple of `config.attention_window`: 1024
 82%|████████▏ | 42/51 [1:22:30<17:39, 117.73s/it]Input ids are automatically padded from 215 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 539 to 1024 to be a multiple of `config.attention_window`: 1024
 84%|████████▍ | 43/51 [1:24:26<15:38, 117.27s/it]Input ids are automatically padded from 131 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 59 to 1024 to be a multiple of `config.attention_window`: 1024
 86%|████████▋ | 44/51 [1:26:23<13:40, 117.15s/it]Input ids are automatically padded from 117 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 239 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are autom

{'loss': 2.067, 'learning_rate': 5e-06, 'epoch': 0.98}


Input ids are automatically padded from 167 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 446 to 1024 to be a multiple of `config.attention_window`: 1024
100%|██████████| 51/51 [1:39:59<00:00, 117.63s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'repetition_penalty': 3.5, 'length_penalty': 0.8, 'no_repeat_ngram_size': 3}


{'train_runtime': 5999.3081, 'train_samples_per_second': 0.137, 'train_steps_per_second': 0.009, 'train_loss': 2.491255376853195, 'epoch': 1.0}
